In [1]:
import statistics
from PIL import ImageFont, ImageDraw, Image  
import numpy as np
import matplotlib.pyplot as plt
import cv2

Transparent Microsoft logo

In [2]:
microsoft = np.ones((310,689,4), dtype=np.uint8)
microsoft[:,:,:] = [0, 0, 0, 0]

microsoft[104:152, 104:152] = [242, 80, 34, 255]
microsoft[157:206, 104:152] = [1, 164, 239, 255]
microsoft[104:152, 157:206] = [127, 186, 0, 255]
microsoft[157:206, 157:206] = [255, 185, 1, 255]

image = Image.fromarray(microsoft)  

draw = ImageDraw.Draw(image)  
   
 # use a truetype font  
font = ImageFont.truetype('input\Segoe UI Bold.ttf', 77)

draw.text((236,100), "Microsoft", font=font)  

image.save("output/microsoft.png")

Color Recognition

In [7]:
# cap = cv2.VideoCapture(0)
cap = cv2.VideoCapture('input/colors.mp4')
_, frame = cap.read()
rows, cols, _ = frame.shape
n = 0

writer = cv2.VideoWriter('output\mamad.mp4', cv2.VideoWriter_fourcc(*'MJPG'), 30, (cols, rows))

while True:
    n+=1
    _, frame_org = cap.read()
    frame_detect = frame_org[(rows//2 - 50):(rows//2 + 50), (cols//2 - 50):(cols//2 + 50)].copy()
    frame_filtered = cv2.blur(frame_org, [15, 15]) 
    cv2.rectangle(frame_filtered, (cols//2 - 50,rows//2 - 50), (cols//2 + 50,rows//2 + 50), 0,8)
    mean = np.mean(frame_detect, axis=(0,1))
    var = statistics.variance(mean)
    if mean[2] > (mean[0] + mean[1])*0.9:
        color = 'Red'
    elif mean[1] > (mean[0] + mean[2])*0.9:
        color = 'Green'
    elif mean[0] > (mean[1] + mean[2])*0.9:
        color = 'Blue'
    elif mean[1] > mean[0]*1.7 and mean[2] > mean[0]*1.7:
        color = 'Yellow'
    elif mean[2] > (mean[0] + 30) and mean[2] > (mean[0] + 30):
        color = 'Orange'
    elif mean[2] > mean[1]*2 and mean[0] > mean[1]*2:
        color = 'Purple'
    elif var < 50 and np.mean(mean) > 100:
        color = 'White'
    elif var < 10 and np.mean(mean) < 60:
        color = 'Black'
    else:
        color = 'unknown'
    # cv2.putText(frame_filtered, str(var), (100,50), cv2.FONT_HERSHEY_SIMPLEX, 0.5, 0, thickness= 1)
    cv2.putText(frame_filtered, color, (150,50), cv2.FONT_HERSHEY_SIMPLEX, 0.7, 0, thickness= 1)
    frame_filtered[(rows//2 - 50):(rows//2 + 50), (cols//2 - 50):(cols//2 + 50)] = frame_detect
    writer.write(frame_filtered)
    cv2.imshow('result', frame_filtered)
    if cv2.waitKey(25) & 0xFF == ord('q'):
        break
    if n == 1641:
        break

writer.release()
cap.release()
cv2.destroyAllWindows()
